In [61]:
# Library Imports
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from sklearn.linear_model import LinearRegression
import pgeocode     # Postal Code GeoDistance

In [2]:
df = pd.read_csv('Data/eBay_ML_Challenge_Dataset_2021_train.tsv', sep='\t', header=0)
df = df.iloc[0:20000]

In [43]:
print(df.columns)
df.nunique(axis=0)

Index(['b2c_c2c', 'seller_id', 'declared_handling_days',
       'acceptance_scan_timestamp', 'shipment_method_id', 'shipping_fee',
       'carrier_min_estimate', 'carrier_max_estimate', 'item_zip', 'buyer_zip',
       'category_id', 'item_price', 'quantity', 'payment_datetime',
       'delivery_date', 'weight', 'weight_units', 'package_size',
       'record_number'],
      dtype='object')


b2c_c2c                          2
seller_id                    12200
declared_handling_days          10
acceptance_scan_timestamp    19730
shipment_method_id              15
shipping_fee                   862
carrier_min_estimate             5
carrier_max_estimate             6
item_zip                      7414
buyer_zip                    10368
category_id                     33
item_price                    3049
quantity                        17
payment_datetime             19994
delivery_date                  631
weight                         181
weight_units                     1
package_size                     5
record_number                20000
dtype: int64

In [53]:
(abs(df['carrier_min_estimate'])).value_counts()

3    12678
2     7313
1        8
6        1
Name: carrier_min_estimate, dtype: int64

In [57]:
##### Data Formatting #######

# Create Feature Dataframe
df_Feat =  pd.DataFrame(np.nan, index=range(df.shape[0]), columns=['Type','Handle','Ship_Method','Min','Max','Dist'])

# Transaction Type (Bussiness to Consumer = 1.5, C2C = .5)
df_Feat['Type'] = (df['b2c_c2c'] == 'B2C').astype(float) + 0.5

# Handling Days
df_Feat['Handle'] = df['declared_handling_days']

# SHipment Method (1:15)
df_Feat['Ship_Method'] = df['shipment_method_id'] + 1

# Min/Max Estimates
df_Feat['Min'] = abs(df['carrier_min_estimate'])
df_Feat['Max'] = abs(df['carrier_max_estimate'])

# Get Postal Code Distance
df_Feat['Dist'] = pd.Series(postal_dist.query_postal_code(df['item_zip'].values, df['buyer_zip'].values))

# df['accept_time'] = np.nan
# df['time_zone'] = np.nan

# # payment to delivery days
# df["delivery_days"] = ""
# for iRow in range(df.shape[0]):
#     payment = datetime.strptime(df.iloc[iRow]['payment_datetime'][:16], '%Y-%m-%d %H:%M') + timedelta(hours = -int(df.iloc[iRow]['payment_datetime'][-6:-3]))
#     delivery = datetime.strptime(df.iloc[iRow]['delivery_date'],'%Y-%m-%d')
#     difference = delivery - payment
#     df.at[iRow,'delivery_days'] = difference.days



In [58]:
df_Feat[0:100]

,Type,Handle,Ship_Method,Min,Max,Dist
0,1.5,3.0,1,3,5,3001.984391
1,0.5,2.0,1,3,5,NaN
2,1.5,1.0,1,3,5,1102.546134
3,1.5,1.0,1,3,5,1343.238967
4,1.5,1.0,1,3,5,2458.659875
...,...,...,...,...,...,...
95,1.5,0.0,1,3,5,1006.298768
96,1.5,1.0,1,3,5,2685.709509
97,1.5,1.0,1,3,5,1621.530487
98,1.5,1.0,2,2,5,3105.900279
